In [685]:
import os
import numpy as np
import pandas as pd
import matplotlib

In [901]:
in_y = np.abs(np.random.normal(0,10, size = (10000,16)))
in_y = in_y.round(0)
in_y[in_y > 28] = 28
columns_y = ['AX', 'BX', 'BY', 'CY', 'CX', 'DX', 'DY', 'AY', 'AQ', 'AL', 'BQ', 'BL', 'CQ', 'CL', 'DQ', 'DL']
in_y = pd.DataFrame(in_y, columns = columns_y)

In [902]:
in_y.head().astype('int64')

,AX,BX,BY,CY,CX,DX,DY,AY,AQ,AL,BQ,BL,CQ,CL,DQ,DL
0,5,0,1,6,6,21,20,8,8,14,6,2,7,15,6,6
1,6,20,4,3,3,7,3,3,3,7,16,7,1,2,0,11
2,17,6,3,5,15,12,4,14,5,26,16,2,7,7,13,12
3,8,3,7,4,5,4,9,2,2,4,10,7,2,5,17,10
4,3,0,10,2,7,12,7,8,15,11,11,7,1,6,20,1


### Удаление взаимоисключающих элементов  и обратное разложение

In [892]:
def backprop(x, **kwargs):
    for key in kwargs.keys():
        try:
            values = kwargs[key]
            x[values] = x[values] + x[key]/len(values)  
            x[key] = 0
        except KeyError:
            i = len(values)
            while i != 0:
                i -= 1
                x = backprop(x,**{key:values[i]})    
    return x      

In [850]:
def forward(x, **kwargs):
    for key in kwargs.keys():
        values = kwargs[key]
        try:
            minval = np.min(x[values])
            x[values] = x[values] - minval
            x[key] = x[key] + len(values)*minval
        except KeyError:
            i = len(values)
            while i != 0:
                i -= 1
                x = forward(x,**{key:values[i]})       
    return x

In [851]:
def del_pairs(x):
    pairs = ['AXBX','CXDX','BYCY','DYAY']
    for p in pairs:
        if x[p] != 0:
            x[p[1]] =  x[p[1]] + x[p] #direction
            x[p[0]] =  x[p[0]] + x[p] #mass1
            x[p[2]] =  x[p[2]] + x[p] #mass2
            x[p] = 0
    return x 

In [852]:
def round_to_int(x,**kwargs):
    for key in kwargs.keys():
        values = kwargs[key]
        m1 = values[0]
        m2 = values[1]
        koef = [0.25, 0.5]
        for k in koef:
            if (x[m1]-k) == np.floor(x[m1]) or (x[m2]+k) == np.ceil(x[m2]):
                x[m1] = x[m1] - k
                x[m2] = x[m2] + k
            elif (x[m1]+k) == np.ceil(x[m1]) or (x[m2]-k) == np.floor(x[m2]):
                x[m1] = x[m1] + k
                x[m2] = x[m2] - k   
    return x

In [903]:
out_y = in_y.copy()
n1 = in_y.columns[0:8:2]
n2 = in_y.columns[1:8:2]
formula = n1 + n2 + ' = (' + n1 + ' + ' + n2 + ')'
for i in range(4): 
    out_y = out_y.eval(formula[i])
    out_y[[n1[i],n2[i]]] = 0
pairs = {'AXBX':['AX','BX'],'BYCY':['BY','CY'],'CXDX':['CX','DX'],'DYAY':['DY','AY']}
directXY = {'X': ['AXBX','CXDX'],'Y': ['BYCY','DYAY']}
directQL = {'Q': [['AQ','CQ'],['BL','DL']],'L':[['AL','CL'],['BQ','DQ']]}
directQL_sum = {'Q': ['AQ','CQ','BL','DL'],'L':['AL','CL','BQ','DQ']}
massQL = {'A': ['AQ','AL'],'B': ['BQ','BL'],'C': ['CQ','CL'], 'D': ['DQ','DL']}
massXY = {'A': ['AXBX','DYAY'],'B': ['AXBX','BYCY'],'C': ['CXDX','BYCY'], 'D': ['CXDX','DYAY']}
delete = {'del': [['X','Y'],['Q','L']]}
out_y[['A','B','C','D','X','Y','Q','L','del']] = 0
out_y = out_y.apply(del_pairs, axis = 1)
out_y = out_y.apply(backprop, **massQL, axis = 1)
out_y = out_y.apply(forward, **directQL, axis = 1)
out_y = out_y.apply(forward, **delete, axis = 1)
out_y = out_y.apply(forward, **massQL, axis = 1)
out_y = out_y.apply(backprop, **directXY, axis = 1)
out_y = out_y.apply(backprop, **directQL_sum, axis = 1)
out_y = out_y.round()
out_y = out_y.apply(backprop, **massXY, axis = 1)
out_y = out_y.round()
out_y = out_y.apply(backprop, **pairs, axis = 1)
out_y = out_y.drop(columns = ['AXBX','CXDX','BYCY','DYAY','A','B','C','D','del','X','Y','Q','L'])
out_y = out_y.apply(round_to_int, **pairs, axis = 1)
out_y = np.round(out_y)
numb = np.where(out_y>28)[0]
numb = np.unique(numb)
out_y = out_y.drop(numb)

In [1000]:
out_y

,AX,BX,BY,CY,CX,DX,DY,AY,AQ,AL,BQ,BL,CQ,CL,DQ,DL
0,0.0,0.0,1.0,1.0,11.0,11.0,12.0,12.0,0.0,5.0,5.0,0.0,0.0,7.0,5.0,4.0
1,13.0,13.0,2.0,2.0,6.0,6.0,5.0,5.0,5.0,3.0,20.0,5.0,5.0,0.0,0.0,5.0
2,6.0,7.0,0.0,0.0,8.0,8.0,2.0,3.0,0.0,28.0,7.0,0.0,0.0,7.0,7.0,13.0
3,0.0,0.0,0.0,1.0,1.0,1.0,1.0,2.0,1.0,3.0,3.0,0.0,0.0,3.0,7.0,0.0
4,0.0,0.0,1.0,1.0,2.0,2.0,3.0,3.0,8.0,4.0,4.0,0.0,0.0,5.0,19.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,1.0,1.0,0.0,0.0,4.0,4.0,3.0,3.0,10.0,6.0,6.0,0.0,0.0,14.0,6.0,10.0
9996,0.0,0.0,10.0,10.0,3.0,3.0,7.0,7.0,2.0,0.0,4.0,2.0,6.0,0.0,0.0,12.0
9997,0.0,0.0,8.0,8.0,6.0,6.0,2.0,2.0,1.0,0.0,0.0,6.0,1.0,3.0,10.0,1.0
9998,4.0,4.0,1.0,1.0,3.0,3.0,0.0,0.0,2.0,16.0,1.0,2.0,14.0,0.0,0.0,2.0


### Извлечение данных в текстовый файл для Ansys

In [997]:
rand_y = out_y
rand_y = rand_y.astype('str')
rand_y['AX'] = 'numb_dele(1) = ' + rand_y['AX']
eof = pd.DataFrame(columns = rand_y.columns)
rand_y.loc[1::2] = '/eof'
rand_y.to_csv('rand_y.txt',index = False, index_label = False)

In [1001]:
rand_y

,AX,BX,BY,CY,CX,DX,DY,AY,AQ,AL,BQ,BL,CQ,CL,DQ,DL
0,numb_dele(1) = 0.0,0.0,1.0,1.0,11.0,11.0,12.0,12.0,0.0,5.0,5.0,0.0,0.0,7.0,5.0,4.0
1,/eof,/eof,/eof,/eof,/eof,/eof,/eof,/eof,/eof,/eof,/eof,/eof,/eof,/eof,/eof,/eof
2,numb_dele(1) = 6.0,7.0,0.0,0.0,8.0,8.0,2.0,3.0,0.0,28.0,7.0,0.0,0.0,7.0,7.0,13.0
3,/eof,/eof,/eof,/eof,/eof,/eof,/eof,/eof,/eof,/eof,/eof,/eof,/eof,/eof,/eof,/eof
4,numb_dele(1) = 0.0,0.0,1.0,1.0,2.0,2.0,3.0,3.0,8.0,4.0,4.0,0.0,0.0,5.0,19.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,numb_dele(1) = 1.0,1.0,0.0,0.0,4.0,4.0,3.0,3.0,10.0,6.0,6.0,0.0,0.0,14.0,6.0,10.0
9996,/eof,/eof,/eof,/eof,/eof,/eof,/eof,/eof,/eof,/eof,/eof,/eof,/eof,/eof,/eof,/eof
9997,numb_dele(1) = 0.0,0.0,8.0,8.0,6.0,6.0,2.0,2.0,1.0,0.0,0.0,6.0,1.0,3.0,10.0,1.0
9998,/eof,/eof,/eof,/eof,/eof,/eof,/eof,/eof,/eof,/eof,/eof,/eof,/eof,/eof,/eof,/eof
